In [1]:
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.stats.api as sms
import pandas as pd

In [2]:
bcf = pd.read_excel("BCF.xlsx")
ps_bart = pd.read_excel("ps_BART.xlsx")

In [3]:
bcf.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [4]:
# Calculate the mean and standard deviation for each column
mean_values = bcf.mean()
std_values = bcf.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE        0.440719
empirical_ATE_MAE         0.324323
empirical_ATE_MAPE        0.189792
empirical_ATE_len         0.888785
empirical_ATE_cover       0.735120
theoretical_CATE_RMSE     1.542730
theoretical_CATE_MAE      1.232614
theoretical_CATE_MAPE     0.379895
theoretical_CATE_len      2.198504
theoretical_CATE_cover    0.527749
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE        0.085832
empirical_ATE_MAE         0.053441
empirical_ATE_MAPE        0.027443
empirical_ATE_len         0.107911
empirical_ATE_cover       0.127626
theoretical_CATE_RMSE     0.652387
theoretical_CATE_MAE      0.534209
theoretical_CATE_MAPE     0.094846
theoretical_CATE_len      0.602347
theoretical_CATE_cover    0.164479
dtype: float64


In [5]:
bcf.head()

,empirical_ATE_RMSE,empirical_ATE_MAE,empirical_ATE_MAPE,empirical_ATE_len,empirical_ATE_cover,theoretical_CATE_RMSE,theoretical_CATE_MAE,theoretical_CATE_MAPE,theoretical_CATE_len,theoretical_CATE_cover
0,0.725630,0.439764,0.209692,0.853601,0.592,2.612644,2.104423,0.563988,2.383984,0.281280
1,0.384188,0.278820,0.163353,0.927275,0.844,1.227177,0.960728,0.332663,2.040968,0.654240
2,0.344773,0.237698,0.150359,0.855503,0.988,1.724829,1.373824,0.453832,2.266505,0.415360
3,0.401042,0.330735,0.206156,1.092543,0.892,1.398823,1.125615,0.316421,2.501573,0.606848
4,0.516242,0.349663,0.192905,0.897596,0.716,2.222732,1.778891,0.478928,2.160195,0.313488


In [6]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
bcf_SE_empirical_ATE_cover = (((0.95 - bcf['empirical_ATE_cover'])**2))
bcf_AE_empirical_ATE_cover = (np.abs(0.95 - bcf['empirical_ATE_cover']))

bcf_SE_theoretical_CATE_cover = (((0.95 - bcf['theoretical_CATE_cover'])**2))
bcf_AE_theoretical_CATE_cover = (np.abs(0.95 - bcf['theoretical_CATE_cover']))

In [7]:
ps_bart.columns

Index(['empirical_ATE_RMSE', 'empirical_ATE_MAE', 'empirical_ATE_MAPE',
       'empirical_ATE_len', 'empirical_ATE_cover', 'theoretical_CATE_RMSE',
       'theoretical_CATE_MAE', 'theoretical_CATE_MAPE', 'theoretical_CATE_len',
       'theoretical_CATE_cover'],
      dtype='object')

In [8]:
# Calculate the mean and standard deviation for each column
mean_values = ps_bart.mean()
std_values = ps_bart.std()

# Print the results
print("Mean of each column:")
print(mean_values)
print("\nStandard deviation of each column:")
print(std_values)

Mean of each column:
empirical_ATE_RMSE        0.282455
empirical_ATE_MAE         0.220565
empirical_ATE_MAPE        0.137809
empirical_ATE_len         1.445926
empirical_ATE_cover       0.993160
theoretical_CATE_RMSE     0.843960
theoretical_CATE_MAE      0.684888
theoretical_CATE_MAPE     0.249336
theoretical_CATE_len      2.469358
theoretical_CATE_cover    0.880167
dtype: float64

Standard deviation of each column:
empirical_ATE_RMSE        0.099411
empirical_ATE_MAE         0.100812
empirical_ATE_MAPE        0.066624
empirical_ATE_len         0.144942
empirical_ATE_cover       0.021229
theoretical_CATE_RMSE     0.322000
theoretical_CATE_MAE      0.257705
theoretical_CATE_MAPE     0.056837
theoretical_CATE_len      0.440176
theoretical_CATE_cover    0.119274
dtype: float64


In [9]:
# Calculate RSE and AE for 'empirical_ATE_cover' and 'theoretical_CATE_cover'
ps_bart_SE_empirical_ATE_cover = (((0.95 - ps_bart['empirical_ATE_cover'])**2))
ps_bart_AE_empirical_ATE_cover = (np.abs(0.95 - ps_bart['empirical_ATE_cover']))

ps_bart_SE_theoretical_CATE_cover = (((0.95 - ps_bart['theoretical_CATE_cover'])**2))
ps_bart_AE_theoretical_CATE_cover = (np.abs(0.95 - ps_bart['theoretical_CATE_cover']))

In [10]:
# Function to check normality using Shapiro-Wilk test
def check_normality(data):
    stat, p = stats.shapiro(data)
    return p > 0.05  # If p > 0.05, data is normally distributed

# Function to check homoscedasticity using Levene's test
def check_homoscedasticity(data1, data2):
    stat, p = stats.levene(data1, data2)
    return p > 0.05  # If p > 0.05, variances are equal



# ATE RMSE

In [11]:
i=0
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_RMSE
Mann-Whitney U Test:
Statistic: 1200.0, p-value: 1.637606880481271e-20
Wilcoxon Signed-Rank Test:
Statistic: 33.0, p-value: 1.0501887534614503e-17
Kruskal-Wallis H Test:
Statistic: 86.2089552238806, p-value: 1.6189288417900752e-20
Levene's Test:
Statistic: 1.3936992426207229, p-value: 0.23919763077561432
Brown-Forsythe Test:
Statistic: 1.3936992426207229, p-value: 0.23919763077561432


# ATE MAE

In [12]:
i=1
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAE
Mann-Whitney U Test:
Statistic: 1795.0, p-value: 4.884924366900712e-15
Wilcoxon Signed-Rank Test:
Statistic: 304.0, p-value: 2.2317250841330536e-14
Kruskal-Wallis H Test:
Statistic: 61.32552238805965, p-value: 4.837683951814506e-15
Levene's Test:
Statistic: 20.209150824763523, p-value: 1.1802932090590925e-05
Brown-Forsythe Test:
Statistic: 20.209150824763523, p-value: 1.1802932090590925e-05


# ATE MAPE

In [13]:
i=2
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_MAPE
Mann-Whitney U Test:
Statistic: 2158.0, p-value: 3.841598509614429e-12
Wilcoxon Signed-Rank Test:
Statistic: 575.0, p-value: 2.0177434491871465e-11
Kruskal-Wallis H Test:
Statistic: 48.22068059701485, p-value: 3.8085038215314604e-12
Levene's Test:
Statistic: 33.55428641746225, p-value: 2.6899534200219494e-08
Brown-Forsythe Test:
Statistic: 33.55428641746225, p-value: 2.6899534200219494e-08


# CATE RMSE

In [14]:
i=5
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_RMSE
Mann-Whitney U Test:
Statistic: 1450.0, p-value: 4.2158932483429975e-18
Wilcoxon Signed-Rank Test:
Statistic: 9.0, p-value: 5.112831740936498e-18
Kruskal-Wallis H Test:
Statistic: 75.23880597014931, p-value: 4.170883501782662e-18
Levene's Test:
Statistic: 17.795879529120494, p-value: 3.7411649123949096e-05
Brown-Forsythe Test:
Statistic: 17.795879529120494, p-value: 3.7411649123949096e-05


# CATE MAE

In [15]:
i=6
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAE
Mann-Whitney U Test:
Statistic: 1537.0, p-value: 2.668414920810613e-17
Wilcoxon Signed-Rank Test:
Statistic: 3.0, p-value: 4.2663227307182344e-18
Kruskal-Wallis H Test:
Statistic: 71.59623283582084, p-value: 2.6406033971922174e-17
Levene's Test:
Statistic: 18.333664740856563, p-value: 2.8882570069834104e-05
Brown-Forsythe Test:
Statistic: 18.333664740856563, p-value: 2.8882570069834104e-05


# CATE MAPE

In [16]:
i=7
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_MAPE
Mann-Whitney U Test:
Statistic: 1087.0, p-value: 1.1803121045661764e-21
Wilcoxon Signed-Rank Test:
Statistic: 2.0, p-value: 4.139373908714419e-18
Kruskal-Wallis H Test:
Statistic: 91.41235223880597, p-value: 1.1664604595682962e-21
Levene's Test:
Statistic: 14.648044754809872, p-value: 0.00017370281234688472
Brown-Forsythe Test:
Statistic: 14.648044754809872, p-value: 0.00017370281234688472


# ATE len

In [17]:
i=3
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For empirical_ATE_len
Mann-Whitney U Test:
Statistic: 9984.0, p-value: 4.140464634938016e-34
Wilcoxon Signed-Rank Test:
Statistic: 0.0, p-value: 3.896559845095909e-18
Kruskal-Wallis H Test:
Statistic: 148.30003582089557, p-value: 4.078919521304207e-34
Levene's Test:
Statistic: 6.52864812540848, p-value: 0.011366064934612221
Brown-Forsythe Test:
Statistic: 6.52864812540848, p-value: 0.011366064934612221


# CATE len

In [18]:
i=8
group1 = np.array(ps_bart[ps_bart.columns[i]])
group2 = np.array(bcf[bcf.columns[i]])
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For {ps_bart.columns[i]}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For {ps_bart.columns[i]}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For theoretical_CATE_len
Mann-Whitney U Test:
Statistic: 6873.0, p-value: 4.756614417390974e-06
Wilcoxon Signed-Rank Test:
Statistic: 493.0, p-value: 2.8147507892817304e-12
Kruskal-Wallis H Test:
Statistic: 20.944053731343274, p-value: 4.728933931105256e-06
Levene's Test:
Statistic: 6.17498823569934, p-value: 0.013785873177209657
Brown-Forsythe Test:
Statistic: 6.17498823569934, p-value: 0.013785873177209657


# ATE Cover SE

In [19]:
group1 = np.array(ps_bart_SE_empirical_ATE_cover)
group2 = np.array(bcf_SE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover SE")
    print(f"ATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover SE
ATE Cover RMSE for ps-BART: 0.04805163888984437 and BCF: 0.24959775640017276
Mann-Whitney U Test:
Statistic: 1020.0, p-value: 9.025982686452995e-23
Wilcoxon Signed-Rank Test:
Statistic: 61.0, p-value: 3.5861751904339134e-17
Kruskal-Wallis H Test:
Statistic: 96.5018326148034, p-value: 8.916126839872614e-23
Levene's Test:
Statistic: 137.59411980343145, p-value: 1.8106278141016723e-24
Brown-Forsythe Test:
Statistic: 137.59411980343145, p-value: 1.8106278141016723e-24


# ATE Cover AE

In [20]:
group1 = np.array(ps_bart_AE_empirical_ATE_cover)
group2 = np.array(bcf_AE_empirical_ATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For ATE Cover AE")
    print(f"ATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For ATE Cover AE
ATE Cover MAE for ps-BART: 0.047240000000000046 and BCF: 0.22171999999999997
Mann-Whitney U Test:
Statistic: 1020.0, p-value: 9.025982686452995e-23
Wilcoxon Signed-Rank Test:
Statistic: 69.0, p-value: 4.540171017330806e-17
Kruskal-Wallis H Test:
Statistic: 96.5018326148034, p-value: 8.916126839872614e-23
Levene's Test:
Statistic: 188.20449336327823, p-value: 1.5199740535800282e-30
Brown-Forsythe Test:
Statistic: 188.20449336327823, p-value: 1.5199740535800282e-30


# CATE Cover SE

In [21]:
group1 = np.array(ps_bart_SE_theoretical_CATE_cover)
group2 = np.array(bcf_SE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover SE")
    print(f"CATE Cover RMSE for ps-BART: {np.sqrt(np.mean(group1))} and BCF: {np.sqrt(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover SE
CATE Cover RMSE for ps-BART: 0.13769782152641338 and BCF: 0.4528563814961207
Mann-Whitney U Test:
Statistic: 660.0, p-value: 2.882941462530965e-26
Wilcoxon Signed-Rank Test:
Statistic: 10.0, p-value: 5.269204481546392e-18
Kruskal-Wallis H Test:
Statistic: 112.45142762426134, p-value: 2.845513025203008e-26
Levene's Test:
Statistic: 108.21530980428905, p-value: 1.6913436963268802e-20
Brown-Forsythe Test:
Statistic: 108.21530980428905, p-value: 1.6913436963268802e-20


# CATE Cover AE

In [22]:
group1 = np.array(ps_bart_AE_theoretical_CATE_cover)
group2 = np.array(bcf_AE_theoretical_CATE_cover)
# Check assumptions
normality1 = check_normality(group1)
normality2 = check_normality(group2)
homoscedasticity = check_homoscedasticity(group1, group2)

# Determine which tests to use
if normality1 and normality2 and homoscedasticity:
    # Use parametric tests
    t_test = stats.ttest_ind(group1, group2)
    f_test = stats.f_oneway(group1, group2)
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("T-test:")
    print(f"Statistic: {t_test.statistic}, p-value: {t_test.pvalue}")
    print("F-test:")
    print(f"Statistic: {f_test.statistic}, p-value: {f_test.pvalue}")
else:
    # Use nonparametric tests
    mann_whitney_u_test = stats.mannwhitneyu(group1, group2)
    wilcoxon_signed_rank_test = stats.wilcoxon(group1[:min(len(group1), len(group2))], group2[:min(len(group1), len(group2))])
    kruskal_wallis_h_test = stats.kruskal(group1, group2)
    levene_test = stats.levene(group1, group2)
    brown_forsythe_test = stats.levene(group1, group2, center='median')
    
    print(f"For CATE Cover AE")
    print(f"CATE Cover MAE for ps-BART: {(np.mean(group1))} and BCF: {(np.mean(group2))}")
    print("Mann-Whitney U Test:")
    print(f"Statistic: {mann_whitney_u_test.statistic}, p-value: {mann_whitney_u_test.pvalue}")
    print("Wilcoxon Signed-Rank Test:")
    print(f"Statistic: {wilcoxon_signed_rank_test.statistic}, p-value: {wilcoxon_signed_rank_test.pvalue}")
    print("Kruskal-Wallis H Test:")
    print(f"Statistic: {kruskal_wallis_h_test.statistic}, p-value: {kruskal_wallis_h_test.pvalue}")
    print("Levene's Test:")
    print(f"Statistic: {levene_test.statistic}, p-value: {levene_test.pvalue}")
    print("Brown-Forsythe Test:")
    print(f"Statistic: {brown_forsythe_test.statistic}, p-value: {brown_forsythe_test.pvalue}")

For CATE Cover AE
CATE Cover MAE for ps-BART: 0.09634143999999999 and BCF: 0.42240383999999986
Mann-Whitney U Test:
Statistic: 660.0, p-value: 2.882941462530965e-26
Wilcoxon Signed-Rank Test:
Statistic: 11.0, p-value: 5.430296382634715e-18
Kruskal-Wallis H Test:
Statistic: 112.45142762426134, p-value: 2.845513025203008e-26
Levene's Test:
Statistic: 22.146345047521358, p-value: 4.73753763278233e-06
Brown-Forsythe Test:
Statistic: 22.146345047521358, p-value: 4.73753763278233e-06
